In [1]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
csvpath = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/atlasV8/csv'

In [3]:
df_midbrain_distance = pd.read_csv(os.path.join(csvpath, 'midbrain_distance.csv'))
df_midbrain_allen = pd.read_csv(os.path.join(csvpath, 'midbrain_allen.csv'))
df_midbrain_atlas = pd.read_csv(os.path.join(csvpath, 'midbrain_atlas.csv'))

In [4]:
df_brainstem_distance = pd.read_csv(os.path.join(csvpath, 'brainstem_distance.csv'))
df_brainstem_allen = pd.read_csv(os.path.join(csvpath, 'brainstem_allen.csv'))
df_brainstem_atlas = pd.read_csv(os.path.join(csvpath, 'brainstem_atlas.csv'))

 That does not preclude starting the new idea but is just to say that somehow we need to get David's attention on what you have done- do a graph
= box plot of the midbrain com distances vs allen (active atlas based)
=box plot of the brainstem vs allen distances (but not using midbrain values to improve the distances)
This should provide an easy visual...


When the tethering is done for the brainstem structures though- he does not want the midbrain metrics to be included ( I don't know though if he means that the alignment is done with midbrain and then the evaluated outputs are only for the brainstem structures...?) your guess is best
So- in any case it was clear that he did not want metrics for midbrain and brainstem structures to be combined.

In [ ]:
midbrain_distances = df_midbrain_distance.distance
brainstem_distances = df_brainstem_distance.distance
fig, ax = plt.subplots()
columns = [midbrain_distances, brainstem_distances]
ax.boxplot(columns, patch_artist=True)
title1 = f"Midbrain distances\nn={len(midbrain_distances)}, min={round(min(midbrain_distances))}"
title1 += f"\nmax={round(max(midbrain_distances))}, mean={round(np.mean(midbrain_distances))}"
title2 = f"Brain stem distances\nn={len(brainstem_distances)}, min={round(min(brainstem_distances))}"
title2 += f"\nmax={round(max(brainstem_distances))}, mean={round(np.mean(brainstem_distances))}"
plt.xticks([1,2], [title1, title2])
plt.show()

In [ ]:
len(brainstem_distances), min(brainstem_distances), max(brainstem_distances), np.mean(brainstem_distances)

In [6]:
for i, row in df_brainstem_distance.iterrows():
    print(f'<tr><td>{row[0]}</td> <td>{round(row[1],2)}</td></tr>')

<tr><td>Sp5O_L</td> <td>97.11</td></tr>
<tr><td>Sp5O_R</td> <td>132.01</td></tr>
<tr><td>DC_L</td> <td>134.63</td></tr>
<tr><td>6N_L</td> <td>138.32</td></tr>
<tr><td>5N_R</td> <td>149.65</td></tr>
<tr><td>VLL_R</td> <td>151.83</td></tr>
<tr><td>LRt_L</td> <td>167.32</td></tr>
<tr><td>5N_L</td> <td>189.11</td></tr>
<tr><td>6N_R</td> <td>204.79</td></tr>
<tr><td>DC_R</td> <td>219.04</td></tr>
<tr><td>7n_R</td> <td>228.57</td></tr>
<tr><td>SNC_R</td> <td>261.94</td></tr>
<tr><td>SNC_L</td> <td>274.29</td></tr>
<tr><td>7n_L</td> <td>276.58</td></tr>
<tr><td>LRt_R</td> <td>312.05</td></tr>
<tr><td>Sp5I_R</td> <td>320.7</td></tr>
<tr><td>Amb_L</td> <td>323.08</td></tr>
<tr><td>Sp5I_L</td> <td>330.99</td></tr>
<tr><td>VLL_L</td> <td>335.02</td></tr>
<tr><td>7N_L</td> <td>363.45</td></tr>
<tr><td>LC_L</td> <td>398.66</td></tr>
<tr><td>RtTg</td> <td>401.44</td></tr>
<tr><td>Amb_R</td> <td>407.88</td></tr>
<tr><td>LC_R</td> <td>420.65</td></tr>
<tr><td>Sp5C_L</td> <td>491.23</td></tr>
<tr><td>7

In [ ]:
xmin = 8000
xmax = 13000
ymin = 2000
ymax = 6000
zmin = 0
zmax = 25*456

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_midbrain_allen[df_midbrain_allen.S == label]
    atlas_subset = df_midbrain_atlas[df_midbrain_atlas.S == label]
    plt.scatter(allen_subset.X, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Y')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('Allen/Altas X Y scatter plot')    
plt.legend()

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_midbrain_allen[df_midbrain_allen.S == label]
    atlas_subset = df_midbrain_atlas[df_midbrain_atlas.S == label]
    plt.scatter(allen_subset.X, allen_subset.Z, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Z, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Z')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    #ax.invert_yaxis()
plt.title('Allen/Altas X Z scatter plot')    
plt.legend()

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_midbrain_allen[df_midbrain_allen.S == label]
    atlas_subset = df_midbrain_atlas[df_midbrain_atlas.S == label]
    plt.scatter(allen_subset.Z, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.Z, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('Z')
    plt.ylabel('Y')
    ax = plt.gca()
    #ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('Allen/Altas Y Z scatter plot')    
plt.legend()

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_brainstem_allen[df_brainstem_allen.S == label]
    atlas_subset = df_brainstem_atlas[df_brainstem_atlas.S == label]
    plt.scatter(allen_subset.X, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Y')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('Allen/Altas X Y scatter plot')    
plt.legend()

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_brainstem_allen[df_brainstem_allen.S == label]
    atlas_subset = df_brainstem_atlas[df_brainstem_atlas.S == label]
    plt.scatter(allen_subset.X, allen_subset.Z, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Z, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Z')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    #ax.invert_yaxis()
plt.title('Allen/Altas X Z scatter plot')    
plt.legend()

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_brainstem_allen[df_brainstem_allen.S == label]
    atlas_subset = df_brainstem_atlas[df_brainstem_atlas.S == label]
    plt.scatter(allen_subset.Z, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.Z, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('Z')
    plt.ylabel('Y')
    ax = plt.gca()
    #ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('Allen/Altas Y Z scatter plot')    
plt.legend()